# <ins>Evaluation of Generator Reactive Power Capability Characteristics in Load Flow Analysis</ins>

By incorporating the reactive power capability characteristics of a generator, the minimum **(Qmin)** and maximum **(Qmax)** reactive power limits can be determined as a function of the generator’s active power output. This directly influences the load flow result, which will be demonstrated in subsequent sections of this tutorial.

The **q_capability_curve_table** provides a structured representation of the generator’s reactive power limits and consists of the following key variables:
  * **id_q_capability_curve** – Identifier for the reactive power capability curve in the **q_capability_curve_table**.  <br>
  * **min_q_mvar** – Minimum reactive power limit (Qmin). <br>
  * **max_q_mvar** – Maximum reactive power limit (Qmax). <br>

To facilitate the integration of these limits, the function **create_q_capability_curve_characteristics_object** is employed. This function transforms the provided reactive power curve data into a characteristic object, applying linear interpolation to construct a continuous function. The resulting characteristic is stored in the **q_capability_curve_characteristic** data structure.

Following this, a diagnostic function is executed to identify and report any inconsistencies within the dataset, ensuring data integrity.

In [ ]:
import pandas as pd
import numpy as np
import pandapower as pp
from pandapower.control.util.auxiliary import create_q_capability_curve_characteristics_object
from pandapower.control.util.diagnostic import q_capability_curve_characteristics_diagnostic

In [ ]:
#Create network
def test_net_for_q_capability_curve(characteristics_data=False):
    net = pp.create_empty_network()
    bus1 = pp.create_bus(net, name="bus1", vn_kv=20., type="b", min_vm_pu=0., max_vm_pu=1.05)
    bus2 = pp.create_bus(net, name="bus2", vn_kv=110., type="b", min_vm_pu=0., max_vm_pu=1.05)
    bus3 = pp.create_bus(net, name="bus3", vn_kv=110., type="b", min_vm_pu=0., max_vm_pu=1.05)

    pp.create_ext_grid(net, bus3, name="External Grid", vm_pu=1.0, va_degree=0.0,max_p_mw=100000, min_p_mw=0, min_q_mvar=-300, max_q_mvar=300,
                       s_sc_max_mva=10000, s_sc_min_mva=8000, rx_max=0.1, rx_min=0.1)
    # create lines
    pp.create_line_from_parameters(net, bus2, bus3, length_km=10,df=1,max_loading_percent=100,vn_kv=110,max_i_ka=0.74,type="ol",
                   r_ohm_per_km=0.0949, x_ohm_per_km =0.38, c_nf_per_km=0.0092, name="Line")
    # create load
    pp.create_load(net, bus3, p_mw=200, q_mvar=180, name="Load", vm_pu=1.0 )

    # create transformer
    pp.create_transformer_from_parameters(net, bus2, bus1, name="110kV/20kV transformer", parallel=1,max_loading_percent=100,sn_mva=210,
    vn_hv_kv=110, vn_lv_kv=20, vk_percent=12.5, vkr_percent=0.01904762, vk0_percent=10, vkr0_percent=0,
                          shift_degree=330, vector_group="YNd11", i0_percent= 0.26, pfe_kw=0,si0_hv_partial=0.5)

